## Imports

In [1]:
import logging
import os
from typing import List, Optional

import hydra
import omegaconf
import pytorch_lightning as pl
import torch
from lightning.pytorch import Callback
from omegaconf import DictConfig, ListConfig

from nn_core.callbacks import NNTemplateCore
from nn_core.common import PROJECT_ROOT
from nn_core.common.utils import enforce_tags, seed_index_everything
from nn_core.model_logging import NNLogger
from nn_core.serialization import NNCheckpointIO

# Force the execution of __init__.py if this file is executed directly.
import tvp  # noqa
from tvp.data.datamodule import MetaData
from tvp.data.datasets.registry import get_dataset
from tvp.task_vectors.task_vectors import TaskVector
from tvp.utils.io_utils import load_model_from_artifact
from tvp.utils.utils import build_callbacks
from torch.nn.utils import vector_to_parameters
from torch.nn.utils import parameters_to_vector

pylogger = logging.getLogger(__name__)

torch.set_float32_matmul_precision("high")

/mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/data/datamodule.py:193: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), config_name="default")


## Configuration

In [2]:
%load_ext autoreload
%autoreload 2

import hydra
from hydra import initialize, compose
from typing import Dict, List

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path=str("../conf"), job_name="playground")

hydra.initialize()

In [3]:
cfg = compose(config_name="task_vectors", overrides=[])

In [4]:
seed_index_everything(cfg)

cfg.core.tags = enforce_tags(cfg.core.get("tags", None))

template_core: NNTemplateCore = NNTemplateCore(
    restore_cfg=cfg.train.get("restore", None),
)
logger: NNLogger = NNLogger(logging_cfg=cfg.train.logging, cfg=cfg, resume_id=template_core.resume_id)

Seed set to 1608637542


2024-05-29 15:45:12 INFO     Setting seed 1608637542 from seeds[0]                         ]8;id=506654;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py\nn_core.common.utils]8;;\:]8;id=125355;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py#107\107]8;;\

                    INFO     Tags: ['dev']                                                  ]8;id=507214;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py\nn_core.common.utils]8;;\:]8;id=552910;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py#96\96]8;;\

                    INFO     Restoring with mode: <None>                                         ]8;id=346224;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/resume.py\nn_core.resume]8;;\:]8;id=130475;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/resume.py#122\122]8;;\

                    INFO     Instantiating <WandbLogger>                                   ]8;id=170108;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/model_logging.py\nn_core.model_logging]8;;\:]8;id=507338;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/model_logging.py#41\41]8;;\

                    ERROR    Failed to detect the name of this notebook, you can set it manually  ]8;id=25814;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/wandb/jupyter.py\wandb.jupyter]8;;\:]8;id=720665;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/wandb/jupyter.py#224\224]8;;\
                             with the WANDB_NOTEBOOK_NAME environment variable to enable code                      
                             saving.                                                                               

wandb: Currently logged in as: dansolombrino (gladia). Use `wandb login --relogin` to force relogin


## Load models

In [5]:
import copy


zeroshot_identifier = f"{cfg.nn.module.model.model_name}_pt"

zeroshot_model = load_model_from_artifact(artifact_path=f"{zeroshot_identifier}:latest", run=logger.experiment)

finetuned_id_fn = lambda dataset: f"{cfg.nn.module.model.model_name}_{dataset}_{cfg.seed_index}:latest"

finetuned_models = {
    dataset: load_model_from_artifact(artifact_path=finetuned_id_fn(dataset), run=logger.experiment)
    for dataset in cfg.task_vectors.to_apply
}

zeroshot_orig_weights = copy.deepcopy(zeroshot_model.state_dict())

2024-05-29 15:45:22 INFO     Loading model from artifact ViT-B-16_pt:latest                   ]8;id=663784;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/io_utils.py\tvp.utils.io_utils]8;;\:]8;id=459697;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/io_utils.py#15\15]8;;\

wandb: Downloading large artifact ViT-B-16_pt:latest, 426.51MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.9


Loading ViT-B-16 pre-trained weights.


2024-05-29 15:45:25 INFO     Loading pretrained ViT-B-16 from OpenAI.                                       ]8;id=355368;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py\root]8;;\:]8;id=790453;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

2024-05-29 15:45:28 INFO     Loading model from artifact ViT-B-16_Cars_0:latest               ]8;id=527597;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/io_utils.py\tvp.utils.io_utils]8;;\:]8;id=234146;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/io_utils.py#15\15]8;;\

...

2024-05-29 15:46:02 INFO     Loading pretrained ViT-B-16 from OpenAI.                                       ]8;id=434389;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py\root]8;;\:]8;id=806390;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

2024-05-29 15:46:04 INFO     Loading model from artifact ViT-B-16_SVHN_0:latest               ]8;id=229256;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/io_utils.py\tvp.utils.io_utils]8;;\:]8;id=261810;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/io_utils.py#15\15]8;;\

wandb: Downloading large artifact ViT-B-16_SVHN_0:latest, 426.51MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.9


Loading ViT-B-16 pre-trained weights.


2024-05-29 15:46:07 INFO     Loading pretrained ViT-B-16 from OpenAI.                                       ]8;id=673489;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py\root]8;;\:]8;id=756732;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/open_clip/factory.py#82\82]8;;\

## Repair

In [6]:
import torch.nn as nn

class ResetConv(nn.Module):
    def __init__(self, conv):
        super().__init__()
        self.out_channels = conv.out_channels
        self.conv = conv
        self.bn = nn.BatchNorm2d(self.out_channels)
        self.rescale = False

    def set_stats(self, goal_mean, goal_var, eps=1e-5):
        self.bn.bias.data = goal_mean
        goal_std = (goal_var + eps).sqrt()
        self.bn.weight.data = goal_std

    def forward(self, x):
        x = self.conv(x)
        if self.rescale:
            x = self.bn(x)
        else:
            self.bn(x)
        return x
    
class ResetLinear(nn.Module):
    def __init__(self, layer):
        # print(f"\n\n[ResetLinear] layer: {layer}\n\n")
        super().__init__()
        self.layer = layer
        self.bn = nn.BatchNorm1d(layer.out_features)
        self.weight = layer.weight
        self.bias = layer.bias

    def set_stats(self, goal_mean, goal_std):
        self.bn.bias.data = goal_mean
        self.bn.weight.data = goal_std

    def forward(self, x):
        # x.shape: [L, N, C] (in torch.nn.BatchNorm1d doc notation)
        x = self.layer(x)

        # match current shape to shape required by BatchNorm1d
        x = x.permute(1, 2, 0)
        # x.shape: [N, C, L] (in torch.nn.BatchNorm1d doc notation)
        
        x = self.bn(x)

        # match current shape to shape required by Linear
        x = x.permute(2, 0, 1)
        # x.shape: [L, N, C] (in torch.nn.BatchNorm1d doc notation)

        return x
    
class ResetLayerNorm(nn.Module):
    def __init__(self, layer):
        super().__init__()
        self.layer = layer
        self.bn = nn.BatchNorm1d(layer.normalized_shape[0])

    def set_stats(self, goal_mean, goal_std):
        self.bn.bias.data = goal_mean
        self.bn.weight.data = goal_std

    def forward(self, x):
        x = self.layer(x)
        return self.bn(x)


def replace_layers(module):
    for name, child in module.named_children():
        if isinstance(child, nn.Conv2d):
            setattr(module, name, ResetConv(child))
        # elif isinstance(child, nn.LayerNorm):
        #     setattr(module, name, ResetLayerNorm(child))
        elif isinstance(child, nn.Linear):
            setattr(module, name, ResetLinear(child))
        else:
            replace_layers(child)


def make_tracked_net(model):
    tracked_model = copy.deepcopy(model)
    replace_layers(tracked_model.model.visual)
    return tracked_model.eval()

In [7]:
# def compute_task_statistics(model, endpoint_models):
#     for m_interp, *endpoint_modules in zip(model_to_repair.modules(), *[model.modules() for model in endpoint_models]):
#         if isinstance(m_interp, (ResetConv, ResetLayerNorm, ResetLinear)):
#             mu_endpoints = torch.stack([m.bn.running_mean for m in endpoint_modules])
#             goal_mean = mu_endpoints.mean(dim=0)
#             var_endpoints = torch.stack([m.bn.running_var for m in endpoint_modules])
#             goal_var = var_endpoints.mean(dim=0)
#             m_interp.set_stats(goal_mean, goal_var)
#             m_interp.rescale = True


#### Wrap and compute stats

In [8]:
dataset = get_dataset(
    cfg.nn.data.train_dataset,
    preprocess_fn=zeroshot_model.train_preprocess,
    location=cfg.nn.data.data_path,
    batch_size=cfg.nn.data.batch_size.train,
)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
def reset_bn_stats(model, epochs, loader):
    """
    Reset batchnorm stats. We use the train loader with data augmentation as this gives better results.
    """
    # resetting stats to baseline first as below is necessary for stability
    for m in model.modules():
        if type(m) == nn.BatchNorm2d:
            m.momentum = None  # use simple average
            m.reset_running_stats()

    # run a single train epoch with augmentations to recalc stats
    model.train()
    for _ in range(epochs):
        with torch.no_grad():
            for batch in loader:
                if isinstance(batch, Dict):
                    input = batch["x"]
                else:
                    input = batch[0]
                    # print(f"[reset_bn_stats] input.shape before model(): {input.shape}")
                out = model(input.cuda())
                # print(f"[reset_bn_stats] out.shape after model()   : {out.shape}")

In [10]:
dataset_name = 'CIFAR100'
finetuned_model = finetuned_models[dataset_name]
tracked_finetuned_model = make_tracked_net(finetuned_models[dataset_name])

In [11]:
finetuned_model.model.visual

VisualTransformer(
  (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
  (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-11): 12 x ResidualAttentionBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (ln_attn): Identity()
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (ln): Identity()
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
  )
  (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [12]:
reset_bn_stats(tracked_finetuned_model.cuda(), 1, dataset.train_loader)

/mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


## Task vectors

In [13]:
tracked_finetuned_model

ImageEncoder(
  (model): CLIP(
    (visual): VisualTransformer(
      (conv1): ResetConv(
        (conv): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
        (bn): BatchNorm2d(768, eps=1e-05, momentum=None, affine=True, track_running_stats=True)
      )
      (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (transformer): Transformer(
        (resblocks): ModuleList(
          (0-11): 12 x ResidualAttentionBlock(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): MultiheadAttention(
              (out_proj): ResetLinear(
                (layer): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
                (bn): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
            )
            (ln_attn): Identity()
            (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (mlp): Sequential(
            

In [14]:
flatten = lambda model: parameters_to_vector(model.parameters()) 

zeroshot_vec = flatten(zeroshot_model)

In [15]:
task_vectors = [TaskVector.from_models(zeroshot_model, finetuned_models[dataset]) for dataset in cfg.task_vectors.to_apply]

In [16]:

def apply_task_vector(model, task_vector):
    model.load_state_dict({k: v + task_vector[k] for k, v in model.state_dict().items()})

### Aggregate task vectors

In [17]:
with torch.no_grad():
    task_vectors = torch.stack([flatten(finetuned_models[dataset]) - zeroshot_vec for dataset in cfg.task_vectors.to_apply])

### Standard task vectors

In [18]:
task_vectors_sum = torch.sum(task_vectors, dim=0)

In [19]:
alpha = 0.8

multi_task_vector = task_vectors_sum / len(task_vectors)

In [20]:
delta_model = copy.deepcopy(zeroshot_model) 
vector_to_parameters(multi_task_vector, delta_model.parameters())

In [21]:
task_equipped_model = copy.deepcopy(zeroshot_model)
apply_task_vector(task_equipped_model, delta_model.state_dict())

In [22]:
classification_head_identifier = f"{cfg.nn.module.model.model_name}_{cfg.nn.data.dataset.dataset_name}_head"
classification_head = load_model_from_artifact(
    artifact_path=f"{classification_head_identifier}:latest", run=logger.experiment
)

model = hydra.utils.instantiate(
    cfg.nn.module, encoder=task_equipped_model, classifier=classification_head, _recursive_=False
)

2024-05-29 15:47:39 INFO     Loading model from artifact ViT-B-16_CIFAR100_head:latest        ]8;id=547084;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/io_utils.py\tvp.utils.io_utils]8;;\:]8;id=696165;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/io_utils.py#15\15]8;;\

wandb:   1 of 1 files downloaded.  
/mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
/mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'classifier' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['classifier'])`.


## Load dataset

In [23]:
seed_index_everything(cfg)

dataset = get_dataset(
    cfg.nn.data.train_dataset,
    preprocess_fn=model.encoder.train_preprocess,
    location=cfg.nn.data.data_path,
    batch_size=cfg.nn.data.batch_size.train,
)

callbacks: List[Callback] = build_callbacks(cfg.train.callbacks, template_core)

storage_dir: str = cfg.core.storage_dir

pylogger.info("Instantiating the <Trainer>")
trainer = pl.Trainer(
    default_root_dir=storage_dir,
    plugins=[NNCheckpointIO(jailing_dir=logger.run_dir)],
    logger=False,
    callbacks=callbacks,
    **cfg.train.trainer,
    
)

Seed set to 1608637542


2024-05-29 15:47:42 INFO     Setting seed 1608637542 from seeds[0]                         ]8;id=323618;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py\nn_core.common.utils]8;;\:]8;id=918782;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/nn_core/common/utils.py#107\107]8;;\

Files already downloaded and verified
Files already downloaded and verified


2024-05-29 15:47:43 INFO     Adding callback <EarlyStopping>                                    ]8;id=334515;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/utils.py\tvp.utils.utils]8;;\:]8;id=507214;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/utils.py#146\146]8;;\

                    INFO     Adding callback <ModelCheckpoint>                                  ]8;id=641795;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/utils.py\tvp.utils.utils]8;;\:]8;id=92612;file:///mnt/KS_2TB/PARA/Projects/task-vectors-playground/src/tvp/utils/utils.py#146\146]8;;\

...

                    INFO     IPU available: False, using: 0 IPUs           ]8;id=255163;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\lightning.pytorch.utilities.rank_zero]8;;\:]8;id=527597;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\

INFO: HPU available: False, using: 0 HPUs


                    INFO     HPU available: False, using: 0 HPUs           ]8;id=491560;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\lightning.pytorch.utilities.rank_zero]8;;\:]8;id=836233;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\

INFO: `Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


                    INFO     `Trainer(val_check_interval=1.0)` was         ]8;id=349771;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py\lightning.pytorch.utilities.rank_zero]8;;\:]8;id=650903;file:///mnt/KS_2TB/PARA/Resources/miniconda3/envs/tvp/lib/python3.11/site-packages/lightning_utilities/core/rank_zero.py#63\63]8;;\
                             configured so validation will run at the end                                          
                             of the training epoch..                                                               

## Evaluation

In [24]:
# pylogger.info("Evaluating on the training set")
# trainer.test(model=model, dataloaders=dataset.train_loader)

pylogger.info("Evaluating on the test set!")
trainer.test(model=model, dataloaders=dataset.test_loader)

2024-05-29 15:47:44 INFO     Evaluating on the test set!                                                 ]8;id=780251;file:///tmp/ipykernel_75059/1153275028.py\__main__]8;;\:]8;id=183908;file:///tmp/ipykernel_75059/1153275028.py#4\4]8;;\

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 40/40 [00:06<00:00,  6.23it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         acc/test          │    0.7336000204086304     │
│         loss/test         │     1.041458010673523     │
└───────────────────────────┴───────────────────────────┘

[{'acc/test': 0.7336000204086304, 'loss/test': 1.041458010673523}]